# TensorRider Algorithm Testrun

In [1]:
import numpy as np
import tensorflow as tf
import tensorlayer as tl

In [2]:
sess = tf.InteractiveSession()

# Jump to "LOAD ARRAYS" if you want to load a NPZ file

In [3]:
def rgb2mono(rgb):
    r, g, b = rgb[:,:,0], rgb[:,:,1], rgb[:,:,2]
    # mono = 0.2989 * r + 0.5870 * g + 0.1140 * b
    mono = 0.5 * r + 0.25 * g + 0.25 * b
    return mono

In [4]:
def prepareDataArrays(iterator):
    X = np.zeros(4800)
    y = np.zeros(1)
    recordCounter = 0;

    for string_record in iterator:
        recordCounter += 1

        example = tf.train.Example()
        example.ParseFromString(string_record)
        imageString = (example.features.feature['image'].bytes_list.value[0])
        label = (example.features.feature['label'].int64_list.value[0])

        image = np.fromstring(imageString, dtype=np.uint8)
        image = image.reshape((60, 80, 3))
        image = rgb2mono(image)
        image = image.reshape((4800))

        X = np.vstack((X,image))
        y = np.append(y,label)
        if recordCounter % 100 == 0:
            print(recordCounter,end = '->')

    y = y.reshape((recordCounter + 1,))
    y = np.round(y / 7) #Downsampling
    y = y + 6
    return X, y

In [5]:
trainIterator = tf.python_io.tf_record_iterator(path="train.tfrecords")
valIterator = tf.python_io.tf_record_iterator(path="val.tfrecords")
testIterator = tf.python_io.tf_record_iterator(path="test.tfrecords")

# LOAD Datasets from TFRecords

In [6]:
print("\nTrain...")
X_train, y_train = prepareDataArrays(trainIterator)
print("\nVal...")
X_val, y_val = prepareDataArrays(valIterator)
print("\nTest...")
X_test, y_test = prepareDataArrays(testIterator)


Train...
100->

D:\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  


200->300->400->500->600->700->800->900->1000->1100->1200->1300->1400->1500->1600->1700->1800->1900->2000->2100->2200->2300->2400->2500->2600->2700->2800->2900->3000->3100->3200->3300->3400->3500->3600->3700->3800->3900->4000->4100->4200->4300->4400->4500->4600->4700->4800->4900->5000->5100->5200->5300->5400->5500->5600->5700->5800->5900->6000->6100->6200->6300->6400->6500->6600->6700->6800->6900->7000->7100->7200->7300->7400->7500->7600->7700->7800->7900->8000->8100->8200->8300->8400->8500->8600->8700->8800->8900->9000->9100->9200->9300->9400->9500->9600->9700->9800->9900->10000->10100->10200->10300->10400->10500->10600->10700->10800->10900->11000->11100->11200->11300->11400->11500->11600->11700->11800->11900->12000->12100->12200->12300->12400->12500->12600->12700->12800->12900->13000->13100->13200->13300->13400->13500->
Val...
100->200->300->400->500->600->700->800->900->1000->1100->1200->1300->1400->1500->1600->1700->
Test...
100->200->300->400->500->600->700->800->900->1000->1100->1

# To Save the Loaded Arrays:

In [7]:
np.savez("arrays.npz", xtr = X_train, ytr = y_train, xval = X_val, yval = y_val, xt = X_test, yt = y_test)

# LOAD ARRAYS

In [3]:
npRecall = np.load("arrays.npz")
X_train = npRecall["xtr"]
y_train = npRecall["ytr"]
X_val = npRecall["xval"]
y_val = npRecall["yval"]
X_test = npRecall["xt"]
y_test = npRecall["yt"]

In [4]:
x = tf.placeholder(tf.float32, shape=[None, 4800], name='x')
y_ = tf.placeholder(tf.int64, shape=[None], name='y_')

# Define the Neural Network

In [5]:
network = tl.layers.InputLayer(x, name='input')
# network = tl.layers.DropoutLayer(network, keep=0.8, name='drop1')
network = tl.layers.DenseLayer(network, 2048, tf.nn.relu, name='relu1')
# network = tl.layers.DropoutLayer(network, keep=0.5, name='drop2')
network = tl.layers.DenseLayer(network, 2048, tf.nn.relu, name='relu2')
# network = tl.layers.DropoutLayer(network, keep=0.5, name='drop3')
network = tl.layers.DenseLayer(network, 1024, tf.nn.relu, name='relu3')
network = tl.layers.DenseLayer(network, n_units=13, act=tf.identity, name='output')

[TL] InputLayer  input: (?, 4800)
[TL] DenseLayer  relu1: 2048 relu
[TL] DenseLayer  relu2: 2048 relu
[TL] DenseLayer  relu3: 1024 relu
[TL] DenseLayer  output: 13 identity


In [6]:
y = network.outputs
cost = tl.cost.cross_entropy(y, y_, name='cost')
correct_prediction = tf.equal(tf.argmax(y, 1), y_)
acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
y_op = tf.argmax(tf.nn.softmax(y), 1)

In [7]:
train_params = network.all_params
train_op = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(cost, var_list=train_params)

In [8]:
tl.layers.initialize_global_variables(sess)

In [9]:
network.print_params()

[TL]   param   0: relu1/W:0            (4800, 2048)       float32_ref (mean: -2.6912639441434294e-05, median: -4.182178145129001e-06, std: 0.08797827363014221)   
[TL]   param   1: relu1/b:0            (2048,)            float32_ref (mean: 0.0               , median: 0.0               , std: 0.0               )   
[TL]   param   2: relu2/W:0            (2048, 2048)       float32_ref (mean: 7.742324669379741e-05, median: 8.132979564834386e-05, std: 0.08798182755708694)   
[TL]   param   3: relu2/b:0            (2048,)            float32_ref (mean: 0.0               , median: 0.0               , std: 0.0               )   
[TL]   param   4: relu3/W:0            (2048, 1024)       float32_ref (mean: 1.5352756236097775e-05, median: -2.5950435883714817e-05, std: 0.08791917562484741)   
[TL]   param   5: relu3/b:0            (1024,)            float32_ref (mean: 0.0               , median: 0.0               , std: 0.0               )   
[TL]   param   6: output/W:0           (1024, 13)      

In [10]:
network.print_layers()

[TL]   layer   0: relu1/Relu:0         (?, 2048)          float32
[TL]   layer   1: relu2/Relu:0         (?, 2048)          float32
[TL]   layer   2: relu3/Relu:0         (?, 1024)          float32
[TL]   layer   3: output/Identity:0    (?, 13)            float32


# Start Training

In [11]:
tl.utils.fit(
    sess, network, train_op, cost, X_train, y_train, x, y_, acc=acc, batch_size=50, n_epoch=500, print_freq=5, X_val=X_val, y_val=y_val, eval_train=False)

[TL] Start training the network ...
[TL] Epoch 1 of 500 took 4.404456s
[TL]    val loss: 863.873876
[TL]    val acc: 0.552353
[TL] Epoch 5 of 500 took 3.909627s
[TL]    val loss: 389.066162
[TL]    val acc: 0.522941
[TL] Epoch 10 of 500 took 3.868526s
[TL]    val loss: 219.092220
[TL]    val acc: 0.572941
[TL] Epoch 15 of 500 took 3.937354s
[TL]    val loss: 141.789698
[TL]    val acc: 0.587647
[TL] Epoch 20 of 500 took 3.903558s
[TL]    val loss: 90.574774
[TL]    val acc: 0.600000
[TL] Epoch 25 of 500 took 3.908041s
[TL]    val loss: 86.699259
[TL]    val acc: 0.487059
[TL] Epoch 30 of 500 took 3.895489s
[TL]    val loss: 67.408201
[TL]    val acc: 0.547059
[TL] Epoch 35 of 500 took 3.891697s
[TL]    val loss: 41.641300
[TL]    val acc: 0.602353
[TL] Epoch 40 of 500 took 3.928660s
[TL]    val loss: 29.993563
[TL]    val acc: 0.612941
[TL] Epoch 45 of 500 took 3.924129s
[TL]    val loss: 25.767626
[TL]    val acc: 0.575294
[TL] Epoch 50 of 500 took 3.889425s
[TL]    val loss: 21.94217

[TL]    val acc: 0.592353
[TL] Epoch 455 of 500 took 4.093303s
[TL]    val loss: 2.964803
[TL]    val acc: 0.627059
[TL] Epoch 460 of 500 took 3.882324s
[TL]    val loss: 3.238923
[TL]    val acc: 0.618235
[TL] Epoch 465 of 500 took 3.884013s
[TL]    val loss: 3.103954
[TL]    val acc: 0.639412
[TL] Epoch 470 of 500 took 4.045758s
[TL]    val loss: 3.018092
[TL]    val acc: 0.622353
[TL] Epoch 475 of 500 took 3.861598s
[TL]    val loss: 2.979404
[TL]    val acc: 0.625882
[TL] Epoch 480 of 500 took 3.894463s
[TL]    val loss: 2.808971
[TL]    val acc: 0.618235
[TL] Epoch 485 of 500 took 3.927776s
[TL]    val loss: 4.200396
[TL]    val acc: 0.581765
[TL] Epoch 490 of 500 took 3.896533s
[TL]    val loss: 3.055398
[TL]    val acc: 0.620000
[TL] Epoch 495 of 500 took 3.915275s
[TL]    val loss: 2.767656
[TL]    val acc: 0.662353
[TL] Epoch 500 of 500 took 3.853869s
[TL]    val loss: 2.701859
[TL]    val acc: 0.637647
[TL] Total training time: 1986.586011s


In [12]:
tl.utils.test(sess, network, acc, X_test, y_test, x, y_, batch_size=None, cost=cost)

[TL] Start testing the network ...
[TL]    test loss: 4.745813
[TL]    test acc: 0.545230


In [13]:
tl.files.save_npz(network.all_params, name='model.npz')
sess.close()

[TL] [*] model.npz saved
